In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
from tqdm import tqdm

In [2]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

airfoils_list = requests.get("http://www.airfoiltools.com/search/airfoils").text
#Get links for each airfoil
airfoil_links = re.findall(r"\/airfoil\/details\?airfoil=[\w-]+", airfoils_list)

#Add domain name to beginning of each link
airfoil_links = list(map(lambda x: "http://www.airfoiltools.com" + x, airfoil_links))
airfoil_dicts = list(map(lambda x: ({"name": x.split("=")[1] ,"link": x}), airfoil_links))

#batches = list(batch(airfoil_dicts, 200))

#Change this as time goes on
checkpoint = 46

#print("Total batches", len(batches))

airfoil_dicts = airfoil_dicts[checkpoint:]

# print("Scraping batch", batchidx)
# print("Batch size", len(airfoil_dicts))

In [3]:
from io import StringIO

def scrape_airfoil(airfoil):
    airfoil_page = requests.get(airfoil["link"]).text
    
    coords_link = f"http://www.airfoiltools.com/airfoil/seligdatfile?airfoil={airfoil['name']}"
    datfile = StringIO(requests.get(coords_link).text)
    
    coords = np.loadtxt(datfile, unpack=True, skiprows=1) #Load airfoil coords into 2d array
    coords = jnp.asarray(coords)
    
    airfoil["coords"] = coords
    
    #Reynolds numbers to fetch
    reynolds_numbers = map(lambda x: int(x) ,[5e4, 1e5, 2e5, 5e5, 1e6])
    combined_polars = []
    for Re in reynolds_numbers:
        polar_link = f"http://www.airfoiltools.com/polar/csv?polar=xf-{airfoil['name']}-{Re}"
        polars = StringIO(requests.get(polar_link).text)
        try:
            polars = pd.read_csv(polars, skiprows=10, dtype=float, on_bad_lines='skip')
        except:
            return None
        #polars = polars.drop(columns=["Top_Xtr", "Bot_Xtr"]) #Drop unnecessary labels
        polars = polars.assign(Re=Re)
        
        columns = ["Alpha", "Re", "Cl", "Cd", "Cdp", "Cm"]
        
        combined_polars.append(polars[columns])
    
    combined_polars = pd.concat(combined_polars)
    airfoil["polars"] = combined_polars
    
    return airfoil

In [4]:
from concurrent.futures import ThreadPoolExecutor, wait

print(f"Getting airfoil coordinates and polars ({len(airfoil_dicts)} airfoils)")

with ThreadPoolExecutor(max_workers=10) as exec:
    result = tqdm(exec.map(scrape_airfoil, airfoil_dicts), total=len(airfoil_dicts))
    airfoil_dicts = list(result)
    
airfoil_dicts = list(filter(lambda x: x is not None, airfoil_dicts))

Getting airfoil coordinates and polars (1638 airfoils)


100%|██████████| 1638/1638 [04:18<00:00,  6.34it/s]


In [ ]:
import jax.scipy as jsci
import optax

print("Fitting airfoils to parameterized geometry")

for airfoil in tqdm(airfoil_dicts):
    
    coords = airfoil["coords"]
    
    #Split the airfoil into top and bottom halves for interpolation
    tophalf = coords[:, :int(coords[0].size/2 + 1)]
    tophalf = jnp.flip(tophalf, axis=1)
    bottomhalf = coords[:, int(coords[0].size/2):]
            
    def shapecost(params):
        B, T, P, C, E, R = params
        
        top_theta = jnp.linspace(1e-3, jnp.pi - 1e-3, 25)
        bottom_theta = jnp.linspace(jnp.pi + 1e-3, 2*jnp.pi - 1e-3, 25)
        
        @jax.jit
        def X(theta):
            return 0.5 + 0.5 * (
                jnp.abs(
                    jnp.cos(theta)
                ) ** B
                / jnp.cos(theta))
        
        @jax.jit
        def Y(theta):
            x = X(theta)

            y = T / 2
            y *= jnp.abs(jnp.sin(theta)) ** B / jnp.sin(theta)
            y *= 1 - x ** P
            y += C * jnp.sin(jnp.pi * x ** E)
            y += R * jnp.sin(2 * jnp.pi * x)

            return y
        
        #Interpolate top and bottom parts of airfoil to get expected Y values
        y_top = jnp.interp(jnp.flip(X(top_theta)), tophalf[0], tophalf[1])
        y_bottom = jnp.interp(X(bottom_theta), bottomhalf[0], bottomhalf[1])
        
        all_theta = jnp.concat((top_theta, bottom_theta))
        all_y = jnp.concat((jnp.flip(y_top), y_bottom))
        
                
        return jnp.mean((all_y - Y(all_theta)) ** 2)
        
        
    
    #Starting params
    params = jnp.array([1.5, 0.2, 3, 0.1, 1, 0])
    
    solver = optax.lbfgs()
    opt_state = solver.init(params)
    value_and_grad = optax.value_and_grad_from_state(shapecost)
        
    for _ in range(10):
                
        cost, grad = value_and_grad(params, state=opt_state)
        
        updates, opt_state = solver.update(
            grad,
            opt_state,
            params,
            grad=grad,
            value=cost,
            value_fn=shapecost
        )

        params = optax.apply_updates(params, updates)
        
    
    airfoil["geometry"] = params
    airfoil["shapeloss"] = shapecost(params)
    
    geometry_columns = ["B", "T", "P", "C", "E", "R"]
    geometry_dict = dict(zip(geometry_columns, airfoil["geometry"].tolist()))
    
    shaped_polar = airfoil["polars"].assign(**geometry_dict)
    
    complete_columns = geometry_columns + ["Alpha", "Re", "Cl", "Cd", "Cdp", "Cm"]
    shaped_polar = shaped_polar[complete_columns]
    
    shaped_polar.to_csv(f"airfoil_data/airfoils_{checkpoint}.csv", index=False)
    
    checkpoint += 1

Fitting airfoils to parameterized geometry


  3%|▎         | 44/1629 [24:24<15:00:02, 34.07s/it]

In [ ]:
# #Add shape information to the polars
# shaped_polars = []

# for airfoil in airfoil_dicts:
#     geometry_columns = ["B", "T", "P", "C", "E", "R"]
#     geometry_dict = dict(zip(geometry_columns, airfoil["geometry"].tolist()))
    
#     shaped_polar = airfoil["polars"].assign(**geometry_dict)
    
#     complete_columns = geometry_columns + ["Alpha", "Re", "Cl", "Cd", "Cdp", "Cm"]
#     shaped_polar = shaped_polar[complete_columns]
    
#     shaped_polars.append(shaped_polar)

# #Complete dataset for airfoil
# dataset = pd.concat(shaped_polars)
# dataset.to_csv(f"airfoils_{batchidx}.csv", index=False)

# dataset.head()

In [ ]:
# if False:
#     B, T, P, C, E, R = params
        
#     thetas = jnp.linspace(1e-3, 2 * jnp.pi - 1e-3, 100)

#     @jax.jit
#     def X(theta):
#         return 0.5 + 0.5 * (
#             jnp.abs(
#                 jnp.cos(theta)
#             ) ** B
#             / jnp.cos(theta))

#     @jax.jit
#     def Y(theta):
#         x = X(theta)

#         y = T / 2
#         y *= jnp.abs(jnp.sin(theta)) ** B / jnp.sin(theta)
#         y *= 1 - x ** P
#         y += C * jnp.sin(jnp.pi * x ** E)
#         y += R * jnp.sin(2 * jnp.pi * x)

#         return y

#     plt.plot(X(thetas), Y(thetas), label="Predicted")

#     plt.legend()

#     plt.plot(coords[0], coords[1], label="Real")
    
#     plt.gca().set_aspect('equal', adjustable='box')

#     plt.show()